In [ ]:
%run ./../data/load-dataset.ipynb
%run ./../word2vec/_load-w2v-model.ipynb
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, RNN, GRU, LSTM, Dense

In [ ]:
ITERS = 3

RANDOM_SEED = 0

In [ ]:
w2v_weights = w2v_model.syn1neg
key2index = w2v_model.wv.key_to_index

In [ ]:
corpus, labels = df[[proc_doc_col, label_col]].T.values
X = np.array([[key2index[token] for token in doc[:MIN_LENGTH]] for doc in corpus])
y = np.array([np.eye(2)[int(label)] for label in labels])
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, random_state=RANDOM_SEED)

In [ ]:
tf.random.set_seed(RANDOM_SEED)
model = Sequential()
model.add(Embedding(input_dim=len(w2v_weights), output_dim=len(w2v_weights.T), 
                    weights=[w2v_weights], trainable=True))
model.add(GRU(100))
model.add(Dense(2, activation=tf.nn.sigmoid))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_samples, train_labels, validation_data=(test_samples, test_labels), epochs=ITERS, batch_size=32)
model.evaluate(train_samples, train_labels, verbose=True), model.evaluate(test_samples, test_labels, verbose=True)

In [ ]:
normalize = 'true'

true_preds, true_labels = [np.argmax(pred) for pred in model.predict(test_samples)], \
                          [np.argmax(label) for label in test_labels]
ConfusionMatrixDisplay.from_predictions(true_labels, true_preds, normalize=normalize,\
                                        cmap=plt.cm.Blues, display_labels=('reliable', 'unreliable'))